<a href="https://colab.research.google.com/github/jeff10-04/jeff_DTSC3020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [4]:
 #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [5]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [16]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    tables = pd.read_html(html)
    for table in tables:
        if table.shape[1] >= 3:
            return flatten_headers(table)
    return pd.DataFrame() # Return empty DataFrame if no suitable table found

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    df.columns = [col.strip() for col in df.columns]
    # Correct column names based on the actual DataFrame
    alpha2_col = 'Alpha-2 code'
    alpha3_col = 'Alpha-3 code'
    numeric_col = 'Numeric'
    country_col = 'Country'


    for col in [alpha2_col, alpha3_col]:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.upper()
    if numeric_col in df.columns:
        df[numeric_col] = pd.to_numeric(df[numeric_col], errors='coerce').astype('Int64') # Use Int64 for nullable integer

    # Drop rows where all key columns are NaN using the correct column names
    subset_cols = [country_col, alpha2_col, alpha3_col, numeric_col]
    return df.dropna(subset=[col for col in subset_cols if col in df.columns], how='all')


def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    if 'Numeric' in df.columns:
        return df.sort_values(by='Numeric', ascending=False).head(top)
    return df.head(top)

In [17]:
# Q1 — Write your answer here

# Fetch the HTML content from the specified URL
url_q1 = "https://www.iban.com/country-codes"
html_q1 = fetch_html(url_q1)

# Read the table using pandas and flatten the headers
df_q1 = q1_read_table(html_q1)

# Clean the DataFrame
df_q1_cleaned = q1_clean(df_q1)

# Sort and get the top 15
df_q1_top15 = q1_sort_top(df_q1_cleaned)

# Print the Top-15 preview
print("Top-15 IBAN Country Codes (sorted by Numeric):")
display(df_q1_top15)

# Save the cleaned data to a CSV file
df_q1_cleaned.to_csv('data_q1.csv', index=False)
print("\nCleaned data saved to data_q1.csv")

Top-15 IBAN Country Codes (sorted by Numeric):


/tmp/ipython-input-1709887363.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


,Country,Alpha-2 code,Alpha-3 code,Numeric
247,Zambia,ZM,ZMB,894
246,Yemen,YE,YEM,887
192,Samoa,WS,WSM,882
244,Wallis and Futuna,WF,WLF,876
240,Venezuela (Bolivarian Republic of),VE,VEN,862
238,Uzbekistan,UZ,UZB,860
237,Uruguay,UY,URY,858
35,Burkina Faso,BF,BFA,854
243,Virgin Islands (U.S.),VI,VIR,850
236,United States of America (the),US,USA,840



Cleaned data saved to data_q1.csv


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [18]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    soup = BeautifulSoup(html, 'lxml')
    items = []
    for athing in soup.select('.athing'):
        item = {}
        item['rank'] = athing.select_one('.rank').get_text(strip=True).replace('.', '')
        item['title'] = athing.select_one('.titleline a').get_text(strip=True)
        item['link'] = athing.select_one('.titleline a')['href']

        subtext = athing.find_next_sibling('tr').select_one('.subtext')
        if subtext:
            score = subtext.select_one('.score')
            item['points'] = score.get_text(strip=True).replace(' points', '') if score else '0'

            # Extract comments - handle 'discuss' links and missing comments
            comments_link = subtext.select('a')[-1] # Last link in subtext is usually comments or 'discuss'
            comments_text = comments_link.get_text(strip=True)
            if 'comment' in comments_text or 'discuss' in comments_text:
                 item['comments'] = comments_text.replace(' comments', '').replace('\xa0', '') if 'comments' in comments_text else '0'
            else:
                item['comments'] = '0'


            user = subtext.select_one('.hnuser')
            item['user'] = user.get_text(strip=True) if user else ''
        else:
            item['points'] = '0'
            item['comments'] = '0'
            item['user'] = ''

        items.append(item)
    return pd.DataFrame(items)


def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    for col in ['rank', 'points', 'comments']:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
    for col in ['title', 'link', 'user']:
        df[col] = df[col].fillna('')
    return df


def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    return df.sort_values(by='points', ascending=False).head(top)

In [19]:
# Q2 — Write your answer here

# Fetch the HTML content from the specified URL
url_q2 = "https://news.ycombinator.com/"
html_q2 = fetch_html(url_q2)

# Parse the items
df_q2 = q2_parse_items(html_q2)

# Clean the DataFrame
df_q2_cleaned = q2_clean(df_q2)

# Sort and get the top 15
df_q2_top15 = q2_sort_top(df_q2_cleaned)

# Print the Top-15 preview
print("Top-15 Hacker News Stories (sorted by points):")
display(df_q2_top15)

# Save the cleaned data to a CSV file
df_q2_cleaned.to_csv('data_q2.csv', index=False)
print("\nCleaned data saved to data_q2.csv")

Top-15 Hacker News Stories (sorted by points):


,rank,title,link,points,comments,user
12,13,"YouTube Removes Windows 11 Bypass Tutorials, C...",https://news.itsfoss.com/youtube-removes-windo...,502,0,WaitWaitWha
8,9,Why I love OCaml (2023),https://mccd.space/posts/ocaml-the-worlds-best/,312,0,art-w
27,28,VLC's Jean-Baptiste Kempf Receives the Europea...,https://fsfe.org/news/2025/news-20251107-01.en...,282,0,kirschner
29,30,James Watson has died,https://www.nytimes.com/2025/11/07/science/jam...,275,0,granzymes
0,1,Why is Zig so cool?,https://nilostolte.github.io/tech/articles/Zig...,239,0,vitalnodo
5,6,Myna: Monospace typeface designed for symbol-h...,https://github.com/sayyadirfanali/Myna,234,0,birdculture
7,8,Ruby Solved My Problem,https://newsletter.masilotti.com/p/ruby-alread...,203,0,joemasilotti
4,5,How did I get here?,https://how-did-i-get-here.net/,187,0,zachlatta
2,3,Becoming a Compiler Engineer,https://rona.substack.com/p/becoming-a-compile...,166,0,lalitkale
1,2,Snapchat open-sources Valdi a cross-platform U...,https://github.com/Snapchat/Valdi,164,0,yehiaabdelm



Cleaned data saved to data_q2.csv
